<h1><center>Travaux Pratiques 2 : classification supervisée</centre></h1>


On va maintenant travailler avec sur une base de données réelle, et très connue : la base Titanic. Seuls 38% des passagers ont survécu a la traversée du Titanic. 

Il s’agit d'être capable de prédire la survie (ou non) des passagers en fonction de différents attributs liés à leur âge, sexe, et aux conditions de voyage. 

Les attributs contenus dans la base sont :

- PassengerId: Unique ID of the passenger
- Survived: Survived (1) or died (0)
- Pclass: Passenger's class (1st, 2nd, or 3rd)
- Passenger’s name
- Passenger’s sex (male/female)
- Passenger’s age
- SibSp: Number of siblings/spouses aboard the Titanic
- Parch: Number of parents/children aboard the Titanic
- Ticket: Ticket number
- Fare: Fare paid for ticket
- Cabin: Cabin number
- Embarked: Where the passenger got on the ship (C - Cherbourg, S - Southampton, Q = Queenstown)

Voici quelques graphiques qui se passent de trop de commentaires pour décrire les données :

![Screen%20Shot%202019-11-16%20at%206.50.54%20PM.png](attachment:Screen%20Shot%202019-11-16%20at%206.50.54%20PM.png)

![Screen%20Shot%202019-11-16%20at%206.52.17%20PM.png](attachment:Screen%20Shot%202019-11-16%20at%206.52.17%20PM.png)

1. Quelle est la variable a expliquer ? Quelles sont les variables explicatives ?

* Variable a expliquer: Survived
* Variable explicatives: Toutes les autres

2. Toutes les variables explicatives vous semblent-elles vraiment pertinentes ? N’y en a-t-il pas que vous pourriez supprimer avant même de commencer l’analyse ? Si oui, lesquelles ?

Variables explicatives pertinentes: 

* Embarked
* Survived
* Pclass
* Sex
* Age
* SibSp
* Parch
* Fare






## A. Importer et préparer les données

A la différence du TD, on va vraiment utiliser une base d’apprentissage (sur laquelle on va ajuster le modèle) et une base de test (sur laquelle on va tester le modèle). On va aléatoirement construire ces deux bases à partir du jeu de données complet Titanic.csv, mais ça, c’est pour plus tard. D’abord, on va préparer les données, autrement dit les nettoyer un peu. On va commencer par importer les librairies dont on aura besoin.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

3. A l'aide des commandes ci-après, chargez le jeu de données complet dans une matrice (ou plus précisément un DataFrame pandas) que vous appellerez Titanic. Combien d’enregistrements et d’attributs a-t-on dans ce jeu de données ? Les commandes sont données ci-après; lisez-les et comprenez-en la syntaxe.

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Titanic.csv to Titanic (1).csv


In [6]:
Titanic=pd.read_csv("Titanic.csv")
print(Titanic.shape)

(891, 12)


4. Supprimez les attributs explicatifs que vous avez identifiés comme inutiles a la question 2 (ou, autrement dit, ne gardez que les autres). Indice : il vous faudra utiliser la propriété iloc d'un DataFrame pandas (par exemple, si on ne veut conserver que la première et la troisième colonne d'un DataFrame pandas X, il faudra écrire : X=X.iloc[:,[0,2]]). Au final, vous devriez conserver 8 attributs.

In [0]:
Titanic=Titanic.iloc[:,[1,2,4,5,6,7,9,11]]

5. Supprimez les enregistrements avec des valeurs indéfinies (NaN), car la plupart des classifieurs que nous allons utiliser ne peuvent pas les gérer (d’autres modèles le peuvent, mais pas celui-là). Pour cela, lancez la commande :

In [0]:
# Suppression des données manquantes
Titanic=Titanic.dropna(axis=0, how='any')

6. Combien d'enregistrements vous reste-t-il ? Indice : pour répondre à la question, utilisez la propriété shape d'un DataFrame pandas (vue plus haut).

In [9]:
print(Titanic.shape)

(712, 8)


7. On prépare maintenant les données. Expliquez ce que fait chacune des lignes ci-après. 

In [0]:
# Codage en quantitatif des attributs "Embarked" et "Sex", codage en qualitatif de l'attribut "Survived"
import numpy as np
Titanic["Sex_cleaned"]=np.where(Titanic["Sex"]=="male",0,1)
Titanic["Embarked_cleaned"]=np.where(Titanic["Embarked"]=="S",0,
                                  np.where(Titanic["Embarked"]=="C",1,
                                           np.where(Titanic["Embarked"]=="Q",2,3)
                                          )
                                 )
Titanic["Survived_cleaned"]=np.where(Titanic["Survived"]==0,"Not Survived","Survived")

* On rajoute un attribut "Sex_cleaned" la valeur 0 pour "male" sinon 1 
* On rajoute un attribut "Embarked_cleaned"  la valeur 0 par S, 1 pour C et 2 pour Q sinon 3
* On rajoute un attribut "Survived_cleaned" la valeur "Not survived" pour 0 sinon "Survived"

8. Avec la fonction head() qui s'applique sur un DataFrame pandas, visualisez le nouveau jeu de données. Combien d’attributs comporte la matrice Titanic à présent ?

In [11]:
Titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Sex_cleaned,Embarked_cleaned,Survived_cleaned
0,0,3,male,22.0,1,0,7.2500,S,0,0,Not Survived
1,1,1,female,38.0,1,0,71.2833,C,1,1,Survived
2,1,3,female,26.0,0,0,7.9250,S,1,0,Survived
3,1,1,female,35.0,1,0,53.1000,S,1,0,Survived
4,0,3,male,35.0,0,0,8.0500,S,0,0,Not Survived


9. Supprimez les attributs explicatifs Sex, Embarked et Survived originaux, et ne gardez que ceux que vous venez de nettoyer. Vérifiez qu'il ne vous reste bien que 8 attributs.

In [0]:
Titanic=Titanic.iloc[:,[1,3,4,5,6,8,9,10]]

In [13]:
Titanic.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_cleaned,Embarked_cleaned,Survived_cleaned
0,3,22.0,1,0,7.2500,0,0,Not Survived
1,1,38.0,1,0,71.2833,1,1,Survived
2,3,26.0,0,0,7.9250,1,0,Survived
3,1,35.0,1,0,53.1000,1,0,Survived
4,3,35.0,0,0,8.0500,0,0,Not Survived


## B. Séparer les données en base d'apprentissage / base de test

Pour mener à bien notre analyse, on va découper nos données en une base d'apprentissage et une base de test (proportions : $\frac{2}{3}$, $\frac{1}{3}$). La division se fait de manière PRESQUE aléatoire, car on va chercher à créer des bases qui contiennent à peu près les mêmes proportions de personnes qui ont survécu ou non que dans la base globale, grâce à l’option Stratify.

10. Ce découpage se fait grâce au code ci-après. Expliquez ce code ligne à ligne.

In [0]:
from sklearn.model_selection import train_test_split
import time
Titanic_train, Titanic_test = train_test_split(Titanic, test_size=1/3, random_state=int(time.time()), stratify=Titanic["Survived_cleaned"])

Création de la base de test "Titanic_test" pour 1/3 et de la base d'apprentissage "Titanic_train" pour 2/3 de façon aléatoire en gardant la même proportion de "survived" et "Not survived" sur l'attribut "Survived_cleaned"

11. Ecrire le code permettant de savoir combien d’enregistrements comporte la base d'apprentissage ? La base de test ? Quelles proportions de personnes ont survécu dans le jeu de données nettoyé "global" ? La base d’apprentissage ? La base de test ?

In [15]:
print(Titanic_test.shape)
print(Titanic_train.shape)
print("nombre de vivants au total",Titanic[Titanic["Survived_cleaned"]=="Survived"].shape[0])
print("nombre de vivants dans Titanic_train",Titanic_train[Titanic_train["Survived_cleaned"]=="Survived"].shape[0])
print("nombre de vivants dans Titanic_test",Titanic_test[Titanic_test["Survived_cleaned"]=="Survived"].shape[0])

(238, 8)
(474, 8)
nombre de vivants au total 288
nombre de vivants dans Titanic_train 192
nombre de vivants dans Titanic_test 96


12. A partir de la matrice Titanic_train, créez une matrice Titanic_TrainX contenant toutes les variables explicatives et un vecteur Titanic_TrainY contenant la variable à expliquer. Faites de même avec la matrice Titanic_test pour obtenir la matrice Titanic_TestX et le vecteur Titanic_TestY.

In [16]:
Titanic_trainX=Titanic_train.iloc[:,[0,1,2,3,4,5,6]]
Titanic_trainY=Titanic_train.iloc[:,[7]]
print(Titanic_trainX)
print(Titanic_trainY)
Titanic_testX=Titanic_test.iloc[:,[0,1,2,3,4,5,6]]
Titanic_testY=Titanic_test.iloc[:,[7]]
print(Titanic_testX)
print(Titanic_testY)

     Pclass   Age  SibSp  Parch      Fare  Sex_cleaned  Embarked_cleaned
309       1  30.0      0      0   56.9292            1                 1
341       1  24.0      3      2  263.0000            1                 0
261       3   3.0      4      2   31.3875            0                 0
406       3  51.0      0      0    7.7500            0                 0
37        3  21.0      0      0    8.0500            0                 0
..      ...   ...    ...    ...       ...          ...               ...
199       2  24.0      0      0   13.0000            1                 0
363       3  35.0      0      0    7.0500            0                 0
530       2   2.0      1      1   26.0000            1                 0
366       1  60.0      1      0   75.2500            1                 1
699       3  42.0      0      0    7.6500            0                 0

[474 rows x 7 columns]
    Survived_cleaned
309         Survived
341         Survived
261         Survived
406     Not Surv

## C. Exemple de classifieur probabiliste : classifieur Bayésien naïf

Comme expliqué dans le cours, le classifieur Bayésien naïf fait une hypothèse d'indépendance des probabilités conditionnelles rarement vérifié en pratique, mais donne en général des résultats acceptables (mais avec une capacité de généralisation assez faible vis-à-vis de nouvelles données). Si nous travaillions toujours sur la base JouerDehors du TD1, nous aurions dû utiliser un classifieur Bayésien naïf faisant l'hypothèse que les variables aléatoires suivent une loi de Bernouilli. Ici, les variables étant quantitatives, nous allons essayer d'appliquer un Bayésien naïf faisant l'hypothèse de lois normales.

13. Les commandes sont données ci-après; lisez-les et comprenez-en la syntaxe (accuracy = taux de classification).

In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Initialisation du classifieur
gnb = GaussianNB()

# Apprentissage (entraînement) du classifieur
gnb.fit(Titanic_trainX, Titanic_trainY)

# Prédiction du classifieur sur la base de test
predictions = gnb.predict(Titanic_testX)

# Estimation du taux de classification sur la base de test (généralisation)
acc=accuracy_score(Titanic_testY,predictions)
print(acc)

0.7815126050420168


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


14. Comparez votre taux de classification avec celui trouvé par votre voisin. Est-elle le même ? Comment l’expliquez-vous ?

différent, c'est lié au random

15. Les performances vous paraissent-elles très bonnes ? Si ce n’est pas le cas, comment l'expliquez-vous ?

C'est le rapport de "bien placés" donc le modèle est valable à 75%. performance convenable mais peut faire mieux.

16. A l’aide de ce même classifieur, donnez les prédictions à partir des données contenues dans la matrice Titanic_TrainX, et comparez ces prédictions avec les sorties réelles contenues dans le vecteur Titanic_TrainY. Donnez le taux de classification (accuracy). Que remarquez-vous ? Comment l'expliquez-vous ?

In [18]:
 # Prédiction du classifieur sur la base de train
predictions1 = gnb.predict(Titanic_trainX)

# Estimation du taux de classification sur la base de train (généralisation)
acc1=accuracy_score(Titanic_trainY,predictions1)
print(acc1)

0.7890295358649789


On va maintenant afficher la matrice de confusion. Attention ! la matrice donnée par le module scikit-learn est inversée par rapport aux notations du cours (prédiction sur les colonnes, réalité sur les lignes), et elle est généralisée au cas multi-classes.
Il faut vraiment faire **très** attention à bien comprendre les notations car tous les logiciels utilisent des représentations différentes ! 

En effet, dans le cours, on avait la notation suivante :
![Screen%20Shot%202019-11-16%20at%207.54.12%20PM.png](attachment:Screen%20Shot%202019-11-16%20at%207.54.12%20PM.png)

Tandis que scikit-learn nous donne (cas à 3 classes 0, 1, 2) :
![Screen%20Shot%202019-11-16%20at%207.54.26%20PM.png](attachment:Screen%20Shot%202019-11-16%20at%207.54.26%20PM.png)

17. Affichez la matrice de confusion en utilisant la commande ci-après, puis réécrire (à la main) la matrice de confusion donnée par Scikit-learn avec les notations du cours (TP, FP, TN, FN), en considérant qu'un survivant est un positif. Indice : ici, Scikit-learn range les classes par ordre alphabétique et donc 'Not Survived' est la classe 0, 'Survived' est la classe 1.

In [19]:
from sklearn.metrics import confusion_matrix
M_confusion=confusion_matrix(Titanic_testY,predictions)
print(M_confusion)
M_confusion=np.array([[M_confusion[1,1],M_confusion[0,1]],[M_confusion[1,0],M_confusion[0,0]]])
print(M_confusion)

[[118  24]
 [ 28  68]]
[[ 68  24]
 [ 28 118]]


18. A partir de la matrice de confusion, vérifiez avec un simple calcul (utilisez le notebook comme calculatrice) que vous obtenez bien le taux de classification (accuracy) de la question 13.

In [20]:
print("accuracy :",(118+68)/(118+24+28+68))

accuracy : 0.7815126050420168


19. Avec un simple calcul (utilisez le notebook comme calculatrice), calculez le rappel et la précision de la classe c='Survived' (détaillez vos calculs).

In [21]:
print("precision :",68/(68+24))
print("rappel :",68/(68+28))


precision : 0.7391304347826086
rappel : 0.7083333333333334


20. Vérifiez votre calcul de précision et de rappel en lançant la commande :

In [22]:
from sklearn.metrics import precision_recall_fscore_support
stats = precision_recall_fscore_support(Titanic_testY,predictions)
print(stats)


(array([0.80821918, 0.73913043]), array([0.83098592, 0.70833333]), array([0.81944444, 0.72340426]), array([142,  96]))


## D. Exemples de classifieurs statistiques

21. On va maintenant essayer avec la méthode des k plus proches voisins, avec k=3. Pour cela, il vous faudra préalablement importer KNeighborsClassifier depuis sklearn.neighbors. Faites-le. Votre taux de classification est-il meilleur ou moins bon qu’avec le classifier naïf de Bayes ? Modifiez la valeur du paramètre k. Arrivez-vous facilement à trouver une valeur de k qui vous semble meilleure ?


In [23]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(Titanic_trainX, Titanic_trainY)

# Prédiction du classifieur sur la base de test
predictions = neigh.predict(Titanic_testX)

# Estimation du taux de classification sur la base de test (généralisation)
acc=accuracy_score(Titanic_testY,predictions)
print(acc) 

0.634453781512605


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


22. On va maintenant essayer avec la méthode SVC (issu de la même famille que le SVR du TP sur la régression, mais cette fois-ci pour de la classification et non de la régression). On va utiliser un SVM avec noyau linéaire.
Le code à exécuter est donné ci-après. Votre taux de classification est-il meilleur ou moins bon qu’avec le classifieur naïf de Bayes ? Le temps d'exécution est-il plus ou moins long ?

## E. Arbres de classification

### Lancer et évaluer des arbres avec différents paramètres

Pour travailler avec des arbres de classification, on va devoir importer la fonction ```DecisionTreeClassifier```, comme suit :
```from sklearn.tree import DecisionTreeClassifier```

Différemment de l'algorithme CART vu en cours, on ne va pas procéder à du post-élagage, mais à de l'élagage en cours de construction de l'arbre. Pour cela nous allons utiliser les deux stratégies données dans les deux sous-sections ci-après.

#### Arbre de décision avec nombre minimum d'exemples par feuille

23. Consultez l'aide de ```DecisionTreeClassifier```, et construisez l'arbre de décision DT1 qui se divise selon la meilleure division à chaque étape selon le critère de Gini, avec un maximum de 10 feuilles au total et un nombre minimum d'enregistrements par feuille fixé à 5, avec une division qui ne peut se faire que si le noeud à diviser contient au moins 2 exemples.

A la manière du classifieur Bayésien naïf ci-dessus, procédez aux différentes étapes :
- Initialisation du classifieur
- Apprentissage (entraînement) du classifieur
- Prédiction du classifieur sur la base de test
- Estimation du taux de classification sur la base de test (généralisation)

24. Notez le taux de classification et comparez-la à celui du classifieur Bayésien naïf.

#### Arbre de décision avec gain d'impureté minimum

25. Consultez l'aide de ```DecisionTreeClassifier```, et construisez l'arbre de décision DT2 qui se divise selon la meilleure division à chaque étape selon le critère de Gini, avec un gain d'impureté minimum (selon le critère de Gini) fixé à 0.008.

A la manière des classifieurs ci-dessus, procédez aux différentes étapes :
- Initialisation du classifieur
- Apprentissage (entraînement) du classifieur
- Prédiction du classifieur sur la base de test
- Estimation du taux de classification sur la base de test (généralisation)

26. Notez le taux de classification et comparez-la à celle du classifieur Bayésien naïf, et à celle du premier arbre de décision.

### Visualisation des arbres de décision

On va maintenant visualiser les deux arbres de décision ci-avant. On va pour cela utiliser pydotplus.

Le code pour DT1 vous est donné ci-après.

In [0]:

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(DT1, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names=Titanic_trainX.columns, class_names=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

27. Déduisez de ce quaphique quelques-unes des règles de classification utilisées par l'arbre.
Quelles sont les variables qui vous semblent les plus discriminantes ?

28. Faites de même pour DT2.

## Forêts aléatoires

29. Ré-exécutez plusieurs fois toutes les cellules précédentes, et notez à chaque fois les taux de classification. 
Ceux-ci sont modifiés à chaque fois qu'on ré-exécute, car la division entre base d'apprentissage et base de test change à chaque fois.

Notez que les taux de classification des arbres de classification changent beaucoup entre deux ré-exécutions. C'est parce que les arbres de décision sont des classifieurs "faibles", au sens où de petits changements dans les données peuvent engendrer de grands changements dans la structure d'arbre.

C'est dans le but d'obtenir un classifieur plus stable qu'on été inventées les **forêts aléatoires**, dont le principe peut être résumé par : "*l'union fait la force*". On parle de **méta-modèle**.

En résumé, au lieu de se contenter d'un seul arbre par nature instable, on va construire une forêt aléatoire constituée de nombreux arbres avec :
 - des versions légèrement modifiées de la base d'apprentissage ;
 - des versions légèrement modifiées des attributs : certains attributs sont aléatoirement écartés de l'analyse pour chacun des arbres de la forêt.
  
Au moment de prendre une décision de classification, on va utiliser un vote à la majorité selon tous ces arbres. Le résultat ainsi obtenu sera plus stable qu'avec un seul arbre (mais pas nécessairement meilleur). Il sera bien sûr plus difficile de visualiser des forêts aléatoires que de simples arbres aléatoires.

Pour travailler avec des forêts aléatoires, on va devoir importer la fonction ```RandomForestClassifier```, comme suit :
```from sklearn.ensemble import RandomForestClassifier```

30. Consultez l'aide de ```RandomForestClassifier```, et construisez la forêt aléatoire constituée de 100 arbres utilisant le critère de Gini et un nombre minimum d'exemples pour découper un noeud fixé à 5.

A la manière des classifieurs ci-dessus, procédez aux différentes étapes :
- Initialisation du classifieur
- Apprentissage (entraînement) du classifieur
- Prédiction du classifieur sur la base de test
- Estimation du taux de classification sur la base de test (généralisation)

31. Notez le taux de classification et comparez-la à celle des classifieurs utilisés précédemment.
Ré-exécutez plusieurs fois cette cellule, et notez que les taux de classification changent à chaque fois. Pourquoi ?

## Validation croisée

Vous n'en avez pas marre de ré-exécuter vos cellules plusieurs fois pour obtenir des résultats qui changent à chaque exécution, et qu'il vous faut marquer sur un bout de papier pour essayer d'en tirer des conclusions sur quel est le meilleur classifieur ? 

Moi, si !

Pour faire cela plus proprement, on va utiliser une stratégie K-fold de vaildation croisée.
L'idée est simple : plutôt que de découper les données en 2 bases : une base d'apprentissage, et une base de test, on va la découper en $K$ bases (folds) de tailles égales (si possible).

On va alors construire $K$ classifieurs. Chacun de ces $K$ classifieurs apprendra sur $K-1$ folds et sera testé sur le fold restant. 
Un avantage supplémentaire de cette procédure (outre le fait que l'on maîtrise mieux le découpage qu'en relançant plusieurs fois la division entre base d'apprentissage et base de test) est que l'on "gâche" moins d'exemples d'apprentissage, d'autant plus que $K$ est élevé.

On va répéter cette procédure plusieurs fois et on obtiendra ainsi ```n_repeats```$\cdot K$ taux de classificaton pour chaque classifieur. Il nous faudra alors calculer la moyenne de tous ces taux de classification, classifieur par classifieur.

Pour cela, vous allez utiliser ```RepeatedKFold``` : ```from sklearn.model_selection import RepeatedKFold```, et lancer ```RepeatedKFold``` avec $K=10$ et ```n_repeats=10```. **Attention à spécifier un bon random_state ! (exemple plus haut dans ce TP)**.

32. Ecrire le code permettant de faire la validation croisée avec RepeatedKFolds.
Indice : il vous faudra utiliser une boucle ```for train, test in <nom de votre RepeatedKFold>.split(Titanic):```, dans laquelle vous allez entraîner chacun des 4 classifieurs vus ci-dessus, estimer leur taux de classification sur la base de test courante et le sauver dans un vecteur numpy de taille ```n_repeats```$\cdot K$. A la fin de la boucle vous pourrez calculer les taux de classification moyens pour chacun des classifieurs.

33. Quel est le classifieur qui vous semble le meilleur ? Est-il plus facile d'en décider avec cette procédure de validation croisée ?

## F. Retour sur la préparation des données

34. Avec une meilleure préparation des données, nous aurions trouvé des résultats complètement différents ; en particulier, 
les SVM auraient été bien meilleurs.
     Avez-vous des idées pour améliorer la préparation des données ?
